# Simulacion de rutas 

### Importar librerias

In [5]:
import pandas as pd
import numpy as np
import math
import random
from geopy.geocoders import Bing

### Primera parte: Distribucion de probabilidad de numero de pedidos

In [6]:
clientes = pd.read_excel("direcciones.xlsx")
clientes

,ciudad,Colonia,codigo postal,Calle,numero de casa
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0
1,Monterrey,VALLE DE SANTA LUCIA MTY,64140,LIC PEDRO ZORRI,404.0
2,Monterrey,JARDINES DEL PAS MTY,64900,AVE FUNDADORES,5300.0
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0
4,Monterrey,VILLA LOS PINOS MTY,64770,PINO SILVESTRE,2923.0
...,...,...,...,...,...
351,Escobedo,LA ALIANZA,64103,PRIV PACHUCA,178.0
352,Escobedo,SANTA LUZ,66070,OXFORD,102.0
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0
354,Escobedo,LOS PARQUES RESIDENCIAL,66023,COBALTO,517.0


In [7]:
informacion_compra = pd.read_excel("informacion_compra.xlsx")
informacion_compra

,Producto,Unidades,Factura
0,48443,1,799186
1,42877,1,717106
2,48296,1,468125
3,34704,1,176490
4,47158,2,674550
...,...,...,...
31649,35303,2,778814
31650,37584,1,632798
31651,19418,1,784747
31652,40176,1,73823


In [8]:
l =  informacion_compra[" Unidades "].mean()#lambda: media de numero de pedidos realizados
l

1.1064320465028117

In [9]:
pedidoMaximo = informacion_compra[" Unidades "].max()

[i for i in range(1,pedidoMaximo + 1)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [10]:
prob = pd.DataFrame({"X": [i for i in range(0,pedidoMaximo + 1)]})


prob["p(x)"] = [ ((l ** x) *  math.exp(-l)) / math.factorial(x) for x in range(0,pedidoMaximo + 1)] 
prob["p(x)"] = prob["p(x)"] / prob["p(x)"].sum()  #normalizamos al dividir entre la suma total de todas las probabilidades

#Probabilidad acumulada
acumul = []
for i in range(0,prob.shape[0]):
    value = prob.loc[0:i, "p(x)"].sum() 
    acumul.append(value)

prob["p(x) acumulada"] = acumul 

#Limite inferior y superior

prob["Lim inferior"] =  [prob["p(x) acumulada"][i] - prob["p(x)"][i]  for i in range(prob.shape[0])]
prob["Lim superior"] =  [prob["p(x) acumulada"][i]  for i in range(prob.shape[0])]


In [11]:
prob

,X,p(x),p(x) acumulada,Lim inferior,Lim superior
0,0,3.307369e-01,0.330737,0.000000,0.330737
1,1,3.659379e-01,0.696675,0.330737,0.696675
2,2,2.024427e-01,0.899118,0.696675,0.899118
3,3,7.466304e-02,0.973781,0.899118,0.973781
4,4,2.065239e-02,0.994433,0.973781,0.994433
5,5,4.570094e-03,0.999003,0.994433,0.999003
6,6,8.427498e-04,0.999846,0.999003,0.999846
7,7,1.332065e-04,0.999979,0.999846,0.999979
8,8,1.842299e-05,0.999997,0.999979,0.999997
9,9,2.264865e-06,1.000000,0.999997,1.000000


### Segunda parte: Simular pedidos de los clientes

In [12]:
numClientes = clientes.shape[0] #Numero de clientes potenciales
numClientes

356

In [13]:
num_pedidos = pd.DataFrame({
    "Cliente": [i for i in range(numClientes)]

})

# Set the "X" column as the index
num_pedidos.set_index("Cliente", inplace=True)

# Reset the index to default integer index and remove the "X" column from the index
num_pedidos.reset_index(drop=True, inplace=True)

#Numero de num_pedidos que pedira cada cliente en base a las probabilidades
numPedidos = []
num_aleatorio = []
for i in range(num_pedidos.shape[0]):
    p = random.uniform(0,1)
    num_aleatorio.append(p)
    for j in range(prob.shape[0]):
        if (prob["Lim inferior"][j] <= p and p < prob["Lim superior"][j]):
            numPedidos.append(j) #Numero de num_pedidos
        else:
            continue

num_pedidos["Aleatorio"] = num_aleatorio
num_pedidos["Numero de pedidos"] = numPedidos

In [14]:
num_pedidos.head()

,Aleatorio,Numero de pedidos
0,0.495452,1
1,0.897858,2
2,0.183679,0
3,0.253825,0
4,0.416331,1


### Parte 3: Designar tipo de pedido por cliente

In [15]:
ventaDia = {}

for idx,producto in enumerate(informacion_compra["Producto"].tolist()):
    if producto not in ventaDia.keys():
        ventaDia[producto] = 1
    else:
        ventaDia[producto] += list(informacion_compra[" Unidades "])[idx]
     

In [16]:
venta_dia =  pd.DataFrame(list(ventaDia.items()), columns=['Producto', 'Venta promedio/dia'])
venta_dia.head()

,Producto,Venta promedio/dia
0,48443,43
1,42877,18
2,48296,28
3,34704,45
4,47158,5


In [17]:
venta_dia["Prob Relativa"] = [venta_dia["Venta promedio/dia"][x] / venta_dia["Venta promedio/dia"].sum() for x in range(venta_dia.shape[0])]

#Probabilidad acumulada
acumul = []
for i in range(0,venta_dia.shape[0]):
    value = venta_dia.loc[0:i, "Prob Relativa"].sum() 
    acumul.append(value)

venta_dia["Prob acumulada"] = acumul 

#Limite inferior y superior

venta_dia["Lim inferior"] =  [venta_dia["Prob acumulada"][i] - venta_dia["Prob Relativa"][i]  for i in range(venta_dia.shape[0])]
venta_dia["Lim superior"] =  [venta_dia["Prob acumulada"][i]  for i in range(venta_dia.shape[0])]


In [18]:
venta_dia.head()

,Producto,Venta promedio/dia,Prob Relativa,Prob acumulada,Lim inferior,Lim superior
0,48443,43,0.001245,0.001245,0.000000,0.001245
1,42877,18,0.000521,0.001766,0.001245,0.001766
2,48296,28,0.000811,0.002576,0.001766,0.002576
3,34704,45,0.001303,0.003879,0.002576,0.003879
4,47158,5,0.000145,0.004024,0.003879,0.004024


### Parte 4: Simular qué tipo de pedido realiza cada cliente

In [19]:
tipo_pedidos = pd.DataFrame({
    "Cliente": [i for i in range(num_pedidos.shape[0])]

})

# Set the "X" column as the index
tipo_pedidos.set_index("Cliente", inplace=True)

# Reset the index to default integer index and remove the "X" column from the index
tipo_pedidos.reset_index(drop=True, inplace=True)

def tipoPedido(num_pedidos,prob_tipoPedido,num_productosTotal):

    tipo_pedido = [0 for i in range(num_productosTotal)]

    for i in range(num_pedidos):

        prob_pedido = random.uniform(0,1)
        #print(prob_pedido)

        for j in range(num_productosTotal):           
            if (prob_tipoPedido["Lim inferior"][j] <= prob_pedido and prob_pedido < prob_tipoPedido["Lim superior"][j]):
                tipo_pedido[j] += 1
            else:
                continue
    
    return tipo_pedido
    

In [20]:
def designarPedidos(num_pedidos,prob_tipoPedido,num_productos):

    numPedidos = list(num_pedidos["Numero de pedidos"]) #lista de numeros de pedidos por cada cliente

    pedidos = {}

    names_products = list(prob_tipoPedido["Producto"])
    for i in range(prob_tipoPedido.shape[0]):
        pedidos[names_products[i]] = []


    for num in numPedidos:
        listaPedidos = tipoPedido(num,prob_tipoPedido,num_productos)
        index = 0
        values = list(pedidos.values())
        for key in pedidos.keys():
            values[index].append(listaPedidos[index]) #numero de pedidos por tipo de pedido
            index += 1
        
        for index,key in enumerate(pedidos):
            pedidos[key] = values[index]
    
    # Sort the dictionary by values (values of the keys)
    sorted_keys_by_value = sorted(pedidos, key=lambda k: str(k).zfill(10))

    # Create a new dictionary with keys sorted by values
    pedidos_sorted = {key: pedidos[key] for key in sorted_keys_by_value}

    pedidosFinal = pd.concat([num_pedidos, pd.DataFrame(pedidos_sorted)], axis=1)
    return pedidosFinal

                
    

In [21]:
productos_porCliente= designarPedidos(num_pedidos,venta_dia,venta_dia.shape[0])
productos_porCliente.head()

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52910,52919,52925,52937,52940,52942,52984,52993,53014,53475
0,0.495452,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.897858,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.183679,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.253825,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.416331,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
productos_porCliente.iloc[:,2:productos_porCliente.shape[1]].loc[355,:].value_counts()

0    4760
1       1
Name: 355, dtype: int64

###  Parte 5: Calcular volumen de carga por cada cliente

In [23]:
productos_volumen = pd.read_csv("info_productos.csv")
productos_volumen.head()

,Producto,Volumen mtro3
0,1,0.001050
1,2,0.026710
2,3,0.020000
3,4,0.056515
4,5,0.025238


In [24]:
productos_porCliente.head()

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52910,52919,52925,52937,52940,52942,52984,52993,53014,53475
0,0.495452,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.897858,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.183679,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.253825,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.416331,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
productosVolumen = {}

for i in range(productos_volumen.shape[0]):
    key = productos_volumen["Producto"][i]
    value = productos_volumen[" Volumen mtro3"][i]
    productosVolumen[key] = value

In [26]:
def calcularVolumen(df,numberProduct_clientes,productos_volumen):

        volumenTotalPorPedidos = []
        for idx,productos in df.iterrows():
                volumen_porCliente = 0
                j = 0
                for numeroDeProductos in productos:
                        try:
                                volumen_porCliente += numeroDeProductos  * productos_volumen[numberProduct_clientes[j]]
                        except:
                                None
                        j += 1

                volumenTotalPorPedidos.append(volumen_porCliente)        



        return volumenTotalPorPedidos      
        


In [27]:
productos_porCliente["Volumen por pedidos (m^3)"] = calcularVolumen(productos_porCliente.iloc[:,2:productos_porCliente.shape[1]],list(productos_porCliente.iloc[:,2:productos_porCliente.shape[1]].columns),productosVolumen)

In [28]:
productos_porCliente.head()

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,0.495452,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055245
1,0.897858,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.027600
2,0.183679,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,0.253825,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,0.416331,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.011129


In [29]:
#volumen total entre todos los clientes
productos_porCliente["Volumen por pedidos (m^3)"].sum()

51.70257018000001

In [30]:
clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0
1,Monterrey,VALLE DE SANTA LUCIA MTY,64140,LIC PEDRO ZORRI,404.0
2,Monterrey,JARDINES DEL PAS MTY,64900,AVE FUNDADORES,5300.0
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0
4,Monterrey,VILLA LOS PINOS MTY,64770,PINO SILVESTRE,2923.0


In [31]:
productos_porCliente.head()

,Aleatorio,Numero de pedidos,23,29,30,60,66,67,89,91,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,0.495452,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055245
1,0.897858,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.027600
2,0.183679,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,0.253825,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,0.416331,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.011129


In [32]:
simulacion_clientes = pd.concat([clientes, productos_porCliente], axis=1)
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0,0.495452,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055245
1,Monterrey,VALLE DE SANTA LUCIA MTY,64140,LIC PEDRO ZORRI,404.0,0.897858,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.027600
2,Monterrey,JARDINES DEL PAS MTY,64900,AVE FUNDADORES,5300.0,0.183679,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
3,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0,0.253825,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,Monterrey,VILLA LOS PINOS MTY,64770,PINO SILVESTRE,2923.0,0.416331,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.011129


### Limpieza y filtracion de pedidos

Como existen clientes que no piden ningun pedido podemos reducir la matriz generada en la simulacion

In [33]:
#Eliminamos los clientes que hicieron 0 pedidos
simulacion_clientes =  simulacion_clientes[simulacion_clientes["Numero de pedidos"] != 0]

In [34]:
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,Monterrey,FIERRO MTY,64590,JESUS M GARZA,4026.0,0.495452,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055245
1,Monterrey,VALLE DE SANTA LUCIA MTY,64140,LIC PEDRO ZORRI,404.0,0.897858,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.027600
4,Monterrey,VILLA LOS PINOS MTY,64770,PINO SILVESTRE,2923.0,0.416331,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.011129
6,Monterrey,FOME 6 F S T D M MTY,64233,CALLE JOSE ANTO,7319.0,0.458839,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.794300
7,Monterrey,URBI VILLA DEL REY,64102,LAGO TITICACA,209.0,0.535506,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.089939


### Coordenadas

In [35]:
def quitar_mty(texto):
    if 'MTY' in texto:
        return texto.replace('MTY', '')
    else:
        return texto

# Aplicar la función a la columna 'Ciudad'
simulacion_clientes['Colonia'] = simulacion_clientes['Colonia'].apply(quitar_mty)

# Mostrar el DataFrame resultante
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52919,52925,52937,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3)
0,Monterrey,FIERRO,64590,JESUS M GARZA,4026.0,0.495452,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055245
1,Monterrey,VALLE DE SANTA LUCIA,64140,LIC PEDRO ZORRI,404.0,0.897858,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.027600
4,Monterrey,VILLA LOS PINOS,64770,PINO SILVESTRE,2923.0,0.416331,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.011129
6,Monterrey,FOME 6 F S T D M,64233,CALLE JOSE ANTO,7319.0,0.458839,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.794300
7,Monterrey,URBI VILLA DEL REY,64102,LAGO TITICACA,209.0,0.535506,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.089939


In [36]:
def obtener_coordenadas(tabla):
    coords = []
    geolocator = Bing(api_key='Ago97Rd3jwxWgFxgUWwgUrZ7S21qlE8ok8UvjoEnaXP2nl2Uh8NHqokudoSjLJzI')
    for i in range(tabla.shape[0]):
        city = tabla.iloc[i,0]
        colony = tabla.iloc[i,1]
        postal_code = tabla.iloc[i,2]
        street_name = tabla.iloc[i,3]
        street_number = tabla.iloc[i,4]

        query = f'{street_name}, {street_number}, {colony}, {postal_code}, {city}, Nuevo León, México'
    
        location = geolocator.geocode(query)

        if location:
            latitude = location.latitude
            longitude = location.longitude
            #print(f'Latitude: {latitude}, Longitude: {longitude}')
            coords.append((latitude,longitude))
        else:
            coords.append((0,0))

    return coords

In [37]:
coordenadas = obtener_coordenadas(simulacion_clientes)
coordenadas[:10]

[(25.68654935, -100.2772068),
 (25.73571083, -100.39369956),
 (25.63611897, -100.28707179),
 (25.79072633, -100.40731977),
 (36.59832921, -121.90723997),
 (25.743958, -100.406852),
 (25.6492493, -100.30286205),
 (25.70853996, -100.37594604),
 (25.66469765, -100.3108902),
 (25.51039989, -100.23189137)]

In [38]:
def return_latitude(x):
    return x[0]

def return_longitude(x):
    return x[1]

In [39]:
simulacion_clientes["Coordenadas"] = coordenadas
simulacion_clientes["Latitud"] = simulacion_clientes["Coordenadas"].apply(return_latitude)
simulacion_clientes["Longitud"] = simulacion_clientes["Coordenadas"].apply(return_longitude)

In [40]:
clientes_index = simulacion_clientes[(simulacion_clientes['Latitud'] < 25) | (simulacion_clientes['Latitud'] > 26)].index
simulacion_clientes = simulacion_clientes.drop(clientes_index)
simulacion_clientes.shape

(221, 4772)

In [41]:
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,Monterrey,FIERRO,64590,JESUS M GARZA,4026.0,0.495452,1,0,0,0,...,0,0,0,0,0,0,0.055245,"(25.68654935, -100.2772068)",25.686549,-100.277207
1,Monterrey,VALLE DE SANTA LUCIA,64140,LIC PEDRO ZORRI,404.0,0.897858,2,0,0,0,...,0,0,0,0,0,0,0.027600,"(25.73571083, -100.39369956)",25.735711,-100.393700
4,Monterrey,VILLA LOS PINOS,64770,PINO SILVESTRE,2923.0,0.416331,1,0,0,0,...,0,0,0,0,0,0,0.011129,"(25.63611897, -100.28707179)",25.636119,-100.287072
6,Monterrey,FOME 6 F S T D M,64233,CALLE JOSE ANTO,7319.0,0.458839,1,0,0,0,...,0,0,0,0,0,0,0.794300,"(25.79072633, -100.40731977)",25.790726,-100.407320
9,Monterrey,CUMBRES 3 SEC,64610,HACIENDA DEL CA,310.0,0.724114,2,0,0,0,...,0,0,0,0,0,0,0.220490,"(25.743958, -100.406852)",25.743958,-100.406852


In [43]:
simulacion_clientes

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,Monterrey,FIERRO,64590,JESUS M GARZA,4026.0,0.495452,1,0,0,0,...,0,0,0,0,0,0,0.055245,"(25.68654935, -100.2772068)",25.686549,-100.277207
1,Monterrey,VALLE DE SANTA LUCIA,64140,LIC PEDRO ZORRI,404.0,0.897858,2,0,0,0,...,0,0,0,0,0,0,0.027600,"(25.73571083, -100.39369956)",25.735711,-100.393700
4,Monterrey,VILLA LOS PINOS,64770,PINO SILVESTRE,2923.0,0.416331,1,0,0,0,...,0,0,0,0,0,0,0.011129,"(25.63611897, -100.28707179)",25.636119,-100.287072
6,Monterrey,FOME 6 F S T D M,64233,CALLE JOSE ANTO,7319.0,0.458839,1,0,0,0,...,0,0,0,0,0,0,0.794300,"(25.79072633, -100.40731977)",25.790726,-100.407320
9,Monterrey,CUMBRES 3 SEC,64610,HACIENDA DEL CA,310.0,0.724114,2,0,0,0,...,0,0,0,0,0,0,0.220490,"(25.743958, -100.406852)",25.743958,-100.406852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Escobedo,BOSQUES DE ESCOB ESC,66072,CEDRO,1034.0,0.777567,2,0,0,0,...,0,0,0,0,0,0,0.040006,"(25.74832044, -100.1695727)",25.748320,-100.169573
349,Escobedo,MISION SAN JUAN,66023,JARDINES DE BAB,109.0,0.752724,2,0,0,0,...,0,0,0,0,0,0,1.179164,"(25.80906296, -100.32276154)",25.809063,-100.322762
350,Escobedo,BUSTAMANTE B,65150,BUGAMBILIAS,435.0,0.637791,1,0,0,0,...,0,0,0,0,0,0,0.163000,"(25.7864574, -100.2671927)",25.786457,-100.267193
353,Escobedo,BUENA VISTA,66583,C PELICANO,238.0,0.796036,2,0,0,0,...,0,0,0,0,0,0,0.002037,"(25.845431, -100.368877)",25.845431,-100.368877


# Exportación de datos

In [42]:
# Export the DataFrame to a CSV file
simulacion_clientes.to_csv('simulacion_clientes2.csv', index=False)  # Set index=False to exclude the DataFrame index